In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [3]:
data = spark.read.csv("/FileStore/tables/customer_churn.csv", 
                      inferSchema=True, 
                      header=True)

In [4]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+

In [5]:
data.columns

Out[ 8 ]: 
['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(inputCols = ['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [8]:
output = assembler.transform(data)

In [9]:
final_data = output.select('features', 'churn')

In [10]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

In [11]:
from pyspark.ml.classification import LogisticRegression

In [12]:
lr_churn = LogisticRegression(labelCol = 'churn')

In [13]:
fitted_churn_model = lr_churn.fit(train_churn)

In [14]:
train_summary = fitted_churn_model.summary

In [15]:
train_summary.predictions.describe().show()

+-------+-------------------+-------------------+
summary| churn| prediction|
+-------+-------------------+-------------------+
 count| 653| 653|
 mean|0.16998468606431852|0.12863705972434916|
 stddev| 0.3759072865863099| 0.3350544479677862|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [17]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [18]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[25.0,9672.03,0.0...| 0|[4.78763843102428...|[0.99173673938388...| 0.0|
[26.0,8787.39,1.0...| 1|[0.63023555732399...|[0.65254287218205...| 0.0|
[29.0,9378.24,0.0...| 0|[4.83895867733346...|[0.99214686770160...| 0.0|
[29.0,11274.46,1....| 0|[4.18777744228197...|[0.98504700051720...| 0.0|
[30.0,6744.87,0.0...| 0|[3.74730749012226...|[0.97696210713236...| 0.0|
[30.0,10183.98,1....| 0|[2.70719769210946...|[0.93745002966666...| 0.0|
[30.0,12788.37,0....| 0|[2.46216764480444...|[0.92144670616133...| 0.0|
[31.0,7073.61,0.0...| 0|[3.35831362988091...|[0.96637602388749...| 0.0|
[31.0,10182.6,1.0...| 0|[4.4886012875217,...|[0.98888850341107...| 0.0|
[31.0,11297.57,1....| 1|[0.83849342275955...|[0.69814781857982...| 0.0|
[31.0,12264.68,1....| 0|[3.24466360824481...|[0.96248088277770...| 0.0|
[32.0,6367.22,1.0...| 0|[2.91536291637039...|[0.94860067653251...| 0.0|
[32.0,10716.75,0....| 0|[4.43197647169269...|[0.98824876918828...| 0.0|
[32.0,12479.72,0....| 0|[4.49926850546890...|[0.98900510593783...| 0.0|
[33.0,5738.82,0.0...| 0|[4.70087983000257...|[0.99099455664685...| 0.0|
[33.0,7750.54,1.0...| 0|[4.17817816230321...|[0.98490494849481...| 0.0|
[33.0,10309.71,1....| 0|[5.91400695906027...|[0.99730593657444...| 0.0|
[34.0,6461.86,1.0...| 0|[4.20985013047083...|[0.98536866127831...| 0.0|
[34.0,7324.32,0.0...| 0|[1.50435180175565...|[0.81822263519324...| 0.0|
[34.0,7818.13,0.0...| 0|[4.06821782652592...|[0.98317990515568...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [19]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', 
                                           labelCol='churn')

In [20]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [21]:
auc

Out[ 30 ]: 0.7291666666666666

In [22]:
### Predict on new data

final_lr_model = lr_churn.fit(final_data)

In [23]:
new_customers = spark.read.csv("/FileStore/tables/new_customers.csv",
                              inferSchema=True,
                              header=True)

In [24]:
new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [25]:
test_new_customers = assembler.transform(new_customers)

In [26]:
test_new_customers.columns

Out[ 37 ]: 
['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'features']

In [27]:
final_result = final_lr_model.transform(test_new_customers)

In [28]:
final_result.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company| features| rawPrediction| probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|[37.0,9935.53,1.0...|[2.22168705251390...|[0.90218018099700...| 0.0|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595046...|[0.00198380445828...| 1.0|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|[-3.7691621189384...|[0.02255110110417...| 1.0|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|[32.0,6487.5,0.0,...|[-5.0956222016539...|[0.00608622642083...| 1.0|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|[32.0,13147.71,1....|[1.10475867224048...|[0.75115067517455...| 0.0|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|[22.0,8445.26,1.0...|[-1.6896019277075...|[0.15582819767621...| 1.0|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+

In [29]:
final_result.select('Company','prediction').show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+

In [30]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Location| Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
 count| 6| 6| 6| 6| 6| 6| 6| 6|
 mean| null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334| null| null|
 stddev| null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785| null| null|
 min|Andrew Mccall| 22.0| 100.0| 0| 1.0| 8.0|085 Austin Views ...|Barron-Robertson|
 max| Taylor Young| 65.0| 13147.71| 1| 10.0| 15.0|Unit 0789 Box 073...| Wood LLC|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+

In [31]:
##https://docs.databricks.com/spark/latest/mllib/binary-classification-mllib-pipelines.html